<a href="https://colab.research.google.com/github/wakepon21/Masa/blob/master/AIQuest_Assesment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importまとめ。pandas-profilingは少々導入が面倒
#!pip install git+https://github.com/pandas-profiling/pandas-profiling.git
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
#import pandas_profiling
import seaborn as sns
#from pandas_profiling.utils.cache import cache_file
%matplotlib inline

# Dataset
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split

# Imbalanced-learn
from imblearn.under_sampling import RandomUnderSampler

#random
import random

# LightGBM
import lightgbm as lgb
#from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error

#Optuna
!pip install optuna
import optuna
from sklearn.metrics import log_loss

warnings.filterwarnings('ignore')

In [ ]:
#train,test,submit_sampleのみっつがそろっているか確認
!ls

In [ ]:
#そろっていなかったら選択
from google.colab import files
train_up = files.upload()

In [ ]:
!ls

In [ ]:
train_base.profile_report()

In [ ]:
X.columns

In [ ]:
sub_base = pd.read_csv("sample_submit.csv",names=("A","B"))
test_base = pd.read_csv("test.csv")
train_base = pd.read_csv("train.csv")
data_base = pd.concat([train_base,test_base], sort=False)
#EDA:整理

#int化

#削除
delete_columns = ['id','amenities','bed_type','room_type',
       'cancellation_policy', 'city', 'cleaning_fee', 'description',
       'first_review', 'host_has_profile_pic', 'host_identity_verified',
       'host_response_rate', 'host_since', 'instant_bookable', 'last_review',
       'latitude', 'longitude', 'name', 'neighbourhood', 
       'property_type', 'review_scores_rating', 'thumbnail_url',
       'zipcode']
data_dropped=data_base.drop(delete_columns, axis=1)
#train["accommodates"]=train["accommodates"].astype(np.int64)

#カテゴリ化

#trainとtestを再度切り分け
train = data_dropped[:len(train)]
test = data_dropped[len(train):]

#train,testを、さらに説明変数Xと、予測変数yに切り分け。y_testは与えられていないのでなし。三種類がでる
y = train['y']
X = train.drop('y', axis = 1)
X_sub = test.copy().drop('y', axis = 1)

In [ ]:
def lgbm_train(X_train_df, X_valid_df, y_train_df, y_valid_df, lgbm_params):
    lgb_train = lgb.Dataset(X_train_df, y_train_df)
    lgb_eval = lgb.Dataset(X_valid_df, y_valid_df, reference=lgb_train)

    # 上記のパラメータでモデルを学習する
    model = lgb.train(lgbm_params, lgb_train,
                      # モデルの評価用データを渡す
                      valid_sets=lgb_eval,
                      # 最大で 1000 ラウンドまで学習する
                      num_boost_round=1000,
                      # 10 ラウンド経過しても性能が向上しないときは学習を打ち切る
                      early_stopping_rounds=10)
    
    return model

In [ ]:
def objective(trial):
    params = {
        'objective': 'regression',
        'max_bin': trial.suggest_int('max_bin', 255, 500),
        'learning_rate': 0.1,
        'num_leaves': trial.suggest_int('num_leaves', 32, 128),
    }
    #LightGBMのデータ分割
    X_train_LGB, X_test_LGB, y_train_LGB, y_test_LGB = train_test_split(X, y, test_size=0.2, random_state=2)
    X_train2_LGB, X_valid_LGB, y_train2_LGB, y_valid_LGB = train_test_split(X, y, test_size=0.2, random_state=4)

    lgb_train = lgb.Dataset(X_train_LGB, y_train_LGB)
    lgb_eval = lgb.Dataset(X_valid_LGB, y_valid_LGB, reference=lgb_train)

    model = lgb.train(params, lgb_train,
                                   valid_sets=[lgb_train, lgb_eval],
                                   verbose_eval=10,
                                   num_boost_round=1000,
                                   early_stopping_rounds=10)

    y_pred_valid_LGB = model.predict(X_valid_LGB, num_iteration=model.best_iteration)
    score = np.sqrt(mean_squared_error(y_valid_LGB, y_pred_valid_LGB))
#    score=y_pred_valid_LGB

    return score

In [ ]:
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, n_trials=30)

In [ ]:
lgbm_params = {
    'learning_rate': 0.05,
    'num_leaves': study.best_params["num_leaves"],
    'boosting_type' : 'gbdt',
    'objective': 'regression',
    'max_bin': study.best_params["max_bin"],
    'max_dapth': 6
}



In [ ]:
def bagging(seed):
    positive_count_train = y_train_LGB.sum()
    X_train2_LGB, X_valid_LGB, y_train2_LGB, y_valid_LGB = train_test_split(X, y, test_size=0.2, random_state=seed)
    model_bagging = lgbm_train(X_train2_LGB, X_valid_LGB, y_train2_LGB, y_valid_LGB, lgbm_params)
    return model_bagging

In [ ]:
%%time
models_LGB = []
for i in range(10):
    models_LGB.append(bagging(i))



In [ ]:
y_preds_LGB = []

for m in models_LGB:
    y_preds_LGB.append(m.predict(X_test_LGB, num_iteration=m.best_iteration))

y_preds_bagging_LGB = sum(y_preds_LGB)/len(y_preds_LGB)
# RMSE を計算する
rmse = np.sqrt(mean_squared_error(y_test_LGB, y_preds_bagging_LGB))
print(rmse)

y_subs_LGB = []
for m in models_LGB:
    y_subs_LGB.append(m.predict(X_sub, num_iteration=m.best_iteration))

y_subs_bagging_LGB = sum(y_subs_LGB)/len(y_subs_LGB)
y_subs_bagging_LGB

In [ ]:
y_sub_LGB = y_subs_bagging_LGB.round()
y_sub_LGB

In [ ]:
sub['B'] = y_sub_LGB
sub.to_csv('submission_LGB.csv', index=False, header=False)
sub.head()
files.download('submission_LGB.csv')